In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

from bs4 import BeautifulSoup 
import requests

url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = requests.get(url).text

soup = BeautifulSoup(html, 'xml')
table = soup.find('table')
rows = table.tbody.find_all("tr")

column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)

for row in rows:
    data=[]
    for td in row.find_all('td'):
        data.append(td.text.strip())
    if len(data)==3:
        df.loc[len(df)] = data

df=df[df['Borough']!='Not assigned']

df["Neighborhood"] = df["Neighborhood"].str.replace("\n","")
df = df.groupby(["Postalcode", "Borough"])["Neighborhood"].apply(", ".join).reset_index()

df.shape

(103, 3)

In [4]:
geo = pd.read_csv('http://cocl.us/Geospatial_data')
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df_mrg = pd.merge(df, geo, how='left', left_on = 'Postalcode', right_on = 'Postal Code')
df_mrg.drop(["Postal Code","Latitude_y", "Longitude_y"], axis=1, inplace=True)
df_mrg.rename(columns={'Latitude_x': 'Latitude', 'Longitude_x': 'Longitude'}, inplace=True)
df_mrg.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
